In [1]:
import pandas as pd
import numpy as np

In [2]:
# reading the data
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
# Store PassengerId out, to use it at the end for Submission
PassengerId = test['PassengerId']

train.sample(frac=0.1, random_state=None)[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
496,497,1,1,"Eustis, Miss. Elizabeth Mussey",female,54.0,1,0,36947,78.2667,D20,C
149,150,0,2,"Byles, Rev. Thomas Roussel Davids",male,42.0,0,0,244310,13.0000,NaN,S
65,66,1,3,"Moubarek, Master. Gerios",male,NaN,1,1,2661,15.2458,NaN,C
828,829,1,3,"McCormack, Mr. Thomas Joseph",male,NaN,0,0,367228,7.7500,NaN,Q
36,37,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C


## Variable Notes
**pclass**: A proxy for socio-economic status (SES)  
1st = Upper  
2nd = Middle  
3rd = Lower  

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...  
**Sibling** = brother, sister, stepbrother, stepsister  
**Spouse** = husband, wife (mistresses and fiancés were ignored)  

parch: The dataset defines family relations in this way...  
Parent = mother, father  
Child = daughter, son, stepdaughter, stepson  
Some children travelled only with a nanny, therefore parch=0 for them.  

In [3]:
# the sample submission file in gender_submission.csv assumes that all female passengers survived (and all male passengers died).
train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [4]:
# Create a naive model estimates that only females survive.
# and check it's accuracy (about 0.79%)
train["Hypo"] = 0 
train.loc[train["Sex"] == "female", "Hypo"] = 1

train["Result"] = 0
train.loc[train["Hypo"] == train["Survived"], "Result"] = 1
print(train["Result"].value_counts(normalize = True))

1    0.786756
0    0.213244
Name: Result, dtype: float64


In [5]:
# Name, Sex, Tickets, Cabin, Embarked(Port of Embarkation) are objetcs
# Age, Cabin, Embarked have null values and Fare(test)
train.drop(['Hypo', 'Result'], axis=1, inplace=True)
train.info(), train.isnull().sum(), test.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


(None, PassengerId      0
 Survived         0
 Pclass           0
 Name             0
 Sex              0
 Age            177
 SibSp            0
 Parch            0
 Ticket           0
 Fare             0
 Cabin          687
 Embarked         2
 dtype: int64, PassengerId      0
 Pclass           0
 Name             0
 Sex              0
 Age             86
 SibSp            0
 Parch            0
 Ticket           0
 Fare             1
 Cabin          327
 Embarked         0
 dtype: int64)

# Feature Engineering

In [6]:
# list of our 2 datasets
datasets = [train, test]

In [7]:
# Name
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

for d in datasets:
    d['Title'] = d['Name'].apply(get_title)

print(pd.crosstab(train['Title'], train['Sex']), '\n')

for d in datasets:
    d['Title'] = d['Title'].replace( ['Capt', 'Col', 'Countess', 'Don',
                                     'Dr', 'Jonkheer', 'Lady', 'Major',
                                     'Rev', 'Sir'], 'Rare' )

    d['Title'] = d['Title'].replace('Mlle', 'Miss')
    d['Title'] = d['Title'].replace('Ms', 'Miss')
    d['Title'] = d['Title'].replace('Mme', 'Mrs')

# Check the title impact on survival rate.
# Miss is the most Survived
print(train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean())

# Mapping titles
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for d in datasets:
    d['Title'] = d['Title'].map(title_mapping)
    d['Title'] = d['Title'].fillna(0)
    d['Title'] = d['Title'].astype(int)

Sex       female  male
Title                 
Capt           0     1
Col            0     2
Countess       1     0
Don            0     1
Dr             1     6
Jonkheer       0     1
Lady           1     0
Major          0     2
Master         0    40
Miss         182     0
Mlle           2     0
Mme            1     0
Mr             0   517
Mrs          125     0
Ms             1     0
Rev            0     6
Sir            0     1 

    Title  Survived
0  Master  0.575000
1    Miss  0.702703
2      Mr  0.156673
3     Mrs  0.793651
4    Rare  0.347826


In [8]:
# Sex
for d in datasets:
    d['Sex'] = d['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [9]:
# Age
for d in datasets:
    d["Age"] = d["Age"].fillna(train["Age"].dropna().median())


train['AgeGroups'] = pd.cut(train['Age'], 5)
print(train[['AgeGroups', "Survived"]].groupby(['AgeGroups'], as_index=False).mean())

for d in datasets:
    d.loc[ d['Age'] <= 16, 'Age'] = 0
    d.loc[ (d['Age'] > 16) & (d['Age'] <= 32), 'Age'] = 1
    d.loc[ (d['Age'] > 32) & (d['Age'] <= 48), 'Age'] = 2
    d.loc[ (d['Age'] > 48) & (d['Age'] <= 64), 'Age'] = 3
    d.loc[ (d['Age'] > 64) & (d['Age'] <= 80), 'Age'] = 4
    
    d["Age"] = d["Age"].astype(int)

          AgeGroups  Survived
0    (0.34, 16.336]  0.550000
1  (16.336, 32.252]  0.344168
2  (32.252, 48.168]  0.404255
3  (48.168, 64.084]  0.434783
4    (64.084, 80.0]  0.090909


In [10]:
## Status
# SibSp and Parch
# sibsp: Sibling(brother, sister, stepbrother, stepsister) or Spouse(husband, wife)
# parch: Parent(mother, father) or Child(daughter, son, stepdaughter, stepson)
# With the number of siblings/spouse and the number of children/parents we can create new feature called Family Size.
for d in datasets:
    d['FamilySize'] = d['SibSp'] + d['Parch'] + 1
print(train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean(), '\n')

for d in datasets:
    d['IsAlone'] = 0
    d.loc[d['FamilySize'] == 1, 'IsAlone'] = 1
train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000 



,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [11]:
# Fare(Passenger fare)
for d in datasets:
    d['Fare'] = d['Fare'].fillna( train['Fare'].median() )

train['FareGroups'] = pd.qcut(train['Fare'], 5)
print(train[['FareGroups', "Survived"]].groupby(['FareGroups'], as_index=False).mean())

for d in datasets:
    d.loc[ d['Fare'] <= 7, 'Fare'] = 0
    d.loc[ (d['Fare'] > 7) & (d['Fare'] <= 10), 'Fare'] = 1
    d.loc[ (d['Fare'] > 10) & (d['Fare'] <= 21), 'Fare'] = 2
    d.loc[ (d['Fare'] > 21) & (d['Fare'] <= 39), 'Fare'] = 3
    d.loc[ (d['Fare'] > 39) & (d['Fare'] <= 512), 'Fare'] = 4
    
    d['Fare'] = d['Fare'].astype(int)

          FareGroups  Survived
0    (-0.001, 7.854]  0.217877
1      (7.854, 10.5]  0.201087
2     (10.5, 21.679]  0.424419
3   (21.679, 39.688]  0.444444
4  (39.688, 512.329]  0.642045


In [12]:
# Cabin(Cabin number)
# train["Cabin"].describe()
for d in datasets:
    d['Has_Cabin'] = d["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

train[['Has_Cabin', 'Survived']].groupby(['Has_Cabin'], as_index=False).mean()

,Has_Cabin,Survived
0,0,0.299854
1,1,0.666667


In [13]:
# Embarked(Port of Embarkation)
# C = Cherbourg, Q = Queenstown, S = Southampton
for d in datasets:
    d['Embarked'] = d['Embarked'].map( {'C':0, 'Q':1, 'S':2} )
    d['Embarked'] = d['Embarked'].fillna(2) # S == 2 (mode)

## Fearture Selection

In [14]:
# Feature Selection
train = train.drop(['AgeGroups', 'FareGroups'], axis = 1)
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch']

train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)

train.sample(frac=0.1, random_state=None)[:5]

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,IsAlone,Has_Cabin
804,1,3,1,1,0,2.0,1,1,1,0
107,1,3,1,1,1,2.0,1,1,1,0
86,0,3,1,0,3,2.0,1,5,0,0
181,0,2,1,1,2,0.0,1,1,1,0
844,0,3,1,1,1,2.0,1,1,1,0


In [15]:
test.sample(frac=0.1, random_state=None)[:5]

,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,IsAlone,Has_Cabin
102,3,1,1,1,1,1,1,1,0
36,3,0,1,1,2,2,1,1,0
137,2,1,1,2,2,1,1,1,0
237,3,1,1,1,0,1,1,1,0
3,3,1,1,1,2,1,1,1,0


In [16]:
train.isna().sum()

Survived      0
Pclass        0
Sex           0
Age           0
Fare          0
Embarked      0
Title         0
FamilySize    0
IsAlone       0
Has_Cabin     0
dtype: int64

# Classifier Comparison
good!  
now we have a clean dataset and ready to predict. let's find which classifier works better on this dataset.

In [17]:
X_train = train.drop("Survived", axis=1)
y_train = train["Survived"]
X_test  = test
X_train.shape, y_train.shape, X_test.shape

((891, 9), (891,), (418, 9))

In [18]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=1, solver='lbfgs')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(round(logreg.score(X_train, y_train)*100, 2))

81.48


In [25]:
# Support Vector Machines
from sklearn.svm import SVC

svc = SVC(kernel='rbf', gamma='auto', C=1)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)

round(svc.score(X_train, y_train)*100, 2)

84.18

In [20]:
# K Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

round(knn.score(X_train, y_train) * 100, 2)

84.51

In [21]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

round(decision_tree.score(X_train, y_train) * 100, 2)

88.89

In [22]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=20)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)

round(random_forest.score(X_train, y_train) * 100, 2)

88.66

most of the classifiers classify the data similarly.  
But i will choose SVC for Submission.

In [26]:
# Generate Submission File 
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': y_pred_svc })
StackingSubmission.to_csv("StackingSubmission.csv", index=False)